# Using `pyfwg` for Direct Morphing

This notebook demonstrates the simplest and most direct way to use the `pyfwg` library: the high-level API functions.

The library provides two main functions for this purpose:
- `morph_epw_global()`: For use with the standard, global version of the FutureWeatherGenerator tool.
- `morph_epw_europe()`: For use with the Europe-specific version of the tool.

This notebook will focus on **`morph_epw_global`**. These functions are designed as one-shot tools that provide full access to all of the tool's parameters without requiring the user to manage the multi-step `MorphingWorkflow` class. They handle parameter validation, temporary file management, and final file placement in a single call.

## Understanding the `morph_epw` Parameters

The functions are powerful because they expose all of the underlying tool's options. Here is a complete reference for all arguments.

### Core Parameters

*   `epw_paths` (`Union[str, List[str]]`): **Required.** The path to a single EPW file (as a string) or a list of paths to multiple EPW files.
*   `fwg_jar_path` (`str`): **Required.** The absolute or relative path to the appropriate `.jar` file.

### Workflow Control Parameters

*   `output_dir` (`str`, default: `'./morphed_epws'`): The directory where the final morphed `.epw` and `.stat` files will be saved.
*   `delete_temp_files` (`bool`, default: `True`): If `True`, the temporary folders are deleted after processing. Set to `False` for debugging.
*   `fwg_show_tool_output` (`bool`, default: `False`): If `True`, prints the FWG tool's console output in real-time. Highly recommended for monitoring progress.

### FutureWeatherGenerator Tool Parameters

*   `fwg_gcms` (`Optional[List[str]]`, default: `None`): A list of GCMs to use. This is specific to the **global** tool (`morph_epw_global`).
*   `fwg_rcm_pairs` (`Optional[List[str]]`, default: `None`): A list of GCM-RCM pairs to use. This is specific to the **europe** tool (`morph_epw_europe`).
*   `fwg_create_ensemble` (`bool`, default: `True`): If `True`, creates an ensemble from the selected models.
*   (All other `fwg_` parameters like `fwg_interpolation_method_id`, `fwg_add_uhi`, etc., are also available. Refer to the library's documentation for a full list.)

## Step 1: Imports and Setup

First, we'll import the necessary functions and set up our file paths. 

**Important:** You must change the `jar_path` variable to the correct location of the `FutureWeatherGenerator_v3.0.1.jar` file on your system. You must also ensure that the `epw_file` path is correct relative to where you are running this notebook.

In [1]:
import os
from pyfwg import morph_epw_global, get_available_lczs, check_lcz_availability, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# Define the path to the EPW file to be processed.
epw_file = 'epws/w_pattern/sevilla_uhi-type-1.epw'

# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar"

## Step 2: Pre-flight Checks (Recommended)

Before running a long morphing process, it's a good practice to validate your parameters. `pyfwg` provides utility functions to help with this.

### Check Available LCZs

The `fwg_epw_original_lcz` and `fwg_target_uhi_lcz` parameters depend on which Local Climate Zones are available for a specific weather file location. We can find this out using `get_available_lczs`.

In [2]:
available_zones = get_available_lczs(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    # In java_class_path_prefix, use 'futureweathergenerator' for the global scope FWG tool, otherwise 'futureweathergenerator_europe'
    java_class_path_prefix='futureweathergenerator',
    show_tool_output=False,
)

# Print the results in a user-friendly format
for filename, lcz_list in available_zones.items():
    print(f"\nAvailable LCZs for: {filename}")
    if lcz_list:
        print(f"  {lcz_list}")
    else:
        print("  Could not be determined.")

2025-09-22 11:19:39 - INFO - --- Fetching available LCZs for 1 EPW file(s) ---
2025-09-22 11:19:39 - INFO - Checking LCZ pair (Original: 0, Target: 0) availability for sevilla_uhi-type-1.epw...
2025-09-22 11:19:39 - INFO - --- Applying UHI effect to sevilla_uhi-type-1.epw ---
2025-09-22 11:19:39 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.UHI_Morph D:\PythonProjects\pyfwg\pyfwg\tutorials\epws\w_pattern\sevilla_uhi-type-1.epw C:\Users\danie\AppData\Local\Temp\tmpxb3pamz9/ true 0:0
2025-09-22 11:19:40 - INFO - Available LCZs for 'sevilla_uhi-type-1.epw': [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]



Available LCZs for: sevilla_uhi-type-1.epw
  [2, 3, 6, 8, 9, 11, 12, 13, 14, 16]


## Step 3: Run the Morphing Process

Now that we have validated our parameters, we can confidently call `morph_epw_global`. We will use the valid LCZ values we identified and a specific GCM for speed.

In [3]:
created_files = morph_epw_global(
    epw_paths=epw_file,
    fwg_jar_path=jar_path,
    output_dir='./custom_output',
    fwg_show_tool_output=True,      # See the tool's progress in real-time
    fwg_gcms=['CanESM5'],           # Use only one specific GCM for speed
    fwg_interpolation_method_id=2,  # Use the "nearest point" method
    fwg_epw_original_lcz=2,         # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3          # Use a validated LCZ from the check above
)

print("\nSuccessfully created files:")
for f in created_files:
    print(f)


2025-09-22 11:20:26 - INFO - --- Starting Direct Global Morphing Process ---
2025-09-22 11:20:26 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-22 11:20:26 - WARNING - Could not map any categories for 'epws/w_pattern/sevilla_uhi-type-1.epw'. Skipping.
2025-09-22 11:20:26 - INFO - Category mapping complete.


AttributeError: 'MorphingWorkflowGlobal' object has no attribute 'set_morphing_config'

## Next Steps

When you need to run multiple morphing configurations or require complex, custom renaming of the output files, the `MorphingWorkflowGlobal` class is the right tool. See the `advanced_workflow_example.ipynb` notebook for a complete guide.